In [1]:
import os
os.chdir('/Users/xinyiwan/Downloads/fintech_545/risk_management')
from library import black_scholes
import library

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import random
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
from scipy.stats import norm
from datetime import datetime
from scipy.optimize import fsolve,minimize

## Problem 1

In [3]:
T = (datetime(2022,4,15)-datetime(2022,3,13)).days/365
S = 165
X = 165
r = 0.0425
q = 0.0053
b = r-q
sigma = 0.2

In [4]:
# Greeks calculated using closed-form formulas
def d1(S,X,b,sigma,T):
    return (np.log(S/X)+(b+sigma**2/2)*T)/sigma/T**0.5
def d2(S,X,b,sigma,T):
    return d1(S,X,b,sigma,T) - sigma*T**0.5
def calculate_delta(S,b,r,T,X,sigma,option):
    if option=='call':
        return np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))
    elif option=='put':
        return np.exp((b-r)*T)*(norm.cdf(d1(S,X,b,sigma,T))-1)
def calculate_gamma(S,b,r,T,X,sigma):
    return norm.pdf(d1(S,X,b,sigma,T))*np.exp((b-r)*T)/(S*sigma*np.sqrt(T))
def calculate_vega(S,b,r,T,X,sigma):
    return S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*np.sqrt(T)
def calculate_theta(S,b,r,T,X,sigma,option):
    if option=='call':
        return -S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*sigma/2/np.sqrt(T)-(b-r)*S*np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))-r*X*np.exp(-r*T)*norm.cdf(d2(S,X,b,sigma,T))
    elif option=='put':
        return -S*np.exp((b-r)*T)*norm.pdf(d1(S,X,b,sigma,T))*sigma/2/np.sqrt(T)+(b-r)*S*np.exp((b-r)*T)*norm.cdf(-d1(S,X,b,sigma,T))+r*X*np.exp(-r*T)*norm.cdf(-d2(S,X,b,sigma,T))
def calculate_rho(S,b,r,T,X,sigma,option):
    if option=='call':
        return T*X*np.exp(-r*T)*norm.cdf(d2(S,X,b,sigma,T))
    elif option=='put':
        return -T*X*np.exp(-r*T)*norm.cdf(-d2(S,X,b,sigma,T))
def calculate_carry_rho(S,b,r,T,X,sigma,option):
    if option=='call':
        return T*S*np.exp((b-r)*T)*norm.cdf(d1(S,X,b,sigma,T))
    elif option=='put':
        return -T*S*np.exp((b-r)*T)*norm.cdf(-d1(S,X,b,sigma,T))

In [5]:
# Greeks calculated through the finite difference method
def fd_delta(S,b,r,T,X,sigma,option,d=0.0001*S):
    if option=='call':
        return (black_scholes(S+d,b,r,T,X,sigma,'call')-black_scholes(S-d,b,r,T,X,sigma,'call'))/2/d
    elif option=='put':
        return (black_scholes(S+d,b,r,T,X,sigma,'put')-black_scholes(S-d,b,r,T,X,sigma,'put'))/2/d
def fd_gamma(S,b,r,T,X,sigma,d=0.0001*S):
    return (black_scholes(S+d,b,r,T,X,sigma,'call')+black_scholes(S-d,b,r,T,X,sigma,'call')-2*black_scholes(S,b,r,T,X,sigma,'call'))/d/d
def fd_vega(S,b,r,T,X,sigma,d=0.0001*S):
    return (black_scholes(S,b,r,T,X,sigma+d,'call')-black_scholes(S,b,r,T,X,sigma-d,'call'))/2/d
def fd_theta(S,b,r,T,X,sigma,option,d=0.0001*S):
    if option=='call':
        res = (black_scholes(S,b,r,T+d,X,sigma,'call')-black_scholes(S,b,r,T-d,X,sigma,'call'))/2/d
        return -res
    elif option=='put':
        res = (black_scholes(S,b,r,T+d,X,sigma,'put')-black_scholes(S,b,r,T-d,X,sigma,'put'))/2/d
        return -res
def fd_rho(S,b,r,T,X,sigma,option,d=0.0001*S):
    if option=='call':
        return (black_scholes(S,b+d,r+d,T,X,sigma,'call')-black_scholes(S,b-d,r-d,T,X,sigma,'call'))/2/d
    elif option=='put':
        return (black_scholes(S,b+d,r+d,T,X,sigma,'put')-black_scholes(S,b-d,r-d,T,X,sigma,'put'))/2/d
def fd_carry_rho(S,b,r,T,X,sigma,option,d=0.0001*S):
    if option=='call':
        return (black_scholes(S,b+d,r,T,X,sigma,'call')-black_scholes(S,b-d,r,T,X,sigma,'call'))/2/d
    elif option=='put':
        return (black_scholes(S,b+d,r,T,X,sigma,'put')-black_scholes(S,b-d,r,T,X,sigma,'put'))/2/d

In [6]:
print("closed form vs finite difference")
print("delta call:", calculate_delta(S,b,r,T,X,sigma,'call'), fd_delta(S,b,r,T,X,sigma,'call')) 
print("delta put:", calculate_delta(S,b,r,T,X,sigma,'put'), fd_delta(S,b,r,T,X,sigma,'put'))
print("gamma:", calculate_gamma(S,b,r,T,X,sigma), fd_gamma(S,b,r,T,X,sigma)) 
print("vega:", calculate_vega(S,b,r,T,X,sigma), fd_vega(S,b,r,T,X,sigma)) 
print("theta call:", calculate_theta(S,b,r,T,X,sigma,'call'), fd_theta(S,b,r,T,X,sigma,'call')) 
print("theta put:", calculate_theta(S,b,r,T,X,sigma,'put'), fd_theta(S,b,r,T,X,sigma,'put'))
print("rho call:", calculate_rho(S,b,r,T,X,sigma,'call'), fd_rho(S,b,r,T,X,sigma,'call')) 
print("rho put:", calculate_rho(S,b,r,T,X,sigma,'put'), fd_rho(S,b,r,T,X,sigma,'put'))
print("carry rho call:", calculate_carry_rho(S,b,r,T,X,sigma,'call'), fd_carry_rho(S,b,r,T,X,sigma,'call')) 
print("carry rho put:", calculate_carry_rho(S,b,r,T,X,sigma,'put'), fd_carry_rho(S,b,r,T,X,sigma,'put'))

closed form vs finite difference
delta call: 0.5340091224850149 0.5340090957299949
delta put: -0.4655118142202754 -0.46551184097585047
gamma: 0.040037930803986446 0.04003792196752335
vega: 19.710179716477544 19.70994837473809
theta call: -24.898522316969515 -24.991087877283185
theta put: -18.786996965277233 -18.879561954174974
rho call: 7.583586080244792 7.583500075884933
rho put: -7.277010958127815 -7.277102474308242
carry rho call: 7.966245676523029 7.966269466580677
carry rho put: -6.944415968299724 -6.944397708631792


In [7]:
def bt_american_cont(S0,X,T,r,q,sigma,N,option):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp((r-q)*dt)-d)/(u-d)
    pd = 1-pu
    discount = np.exp(-r*dt)
    z = 1 if option=='call' else -1
    
    def nNodeFunc(n):
        return (n+1)*(n+2)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)
    optionValues = [0.0]*nNodes

    for j in range(N,-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-X))
            if j < N:
                optionValues[idx] = max(
                    optionValues[idx],
                    discount*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)]))
    return optionValues[0]

In [8]:
def bt_american_discrete(S0, X, T, r, sigma, N, option, dividend_times=None, dividend_amounts=None):
    if dividend_times is None or dividend_amounts is None or (len(dividend_amounts)==0) or (len(dividend_times)==0):
        return bt_american_cont(S0, X, T, r, 0, sigma, N, option)
    elif dividend_times[0] > N:
        return bt_american_cont(S0, X, T, r, 0, sigma, N, option)
    
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1-pu
    discount = np.exp(-r*dt)
    z = 1 if option == 'call' else -1
    
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
   
    nDiv = len(dividend_times)
    nNodes = nNodeFunc(dividend_times[0])
    optionValues = [0.0]*nNodes

    for j in range(dividend_times[0],-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)       
            
            if j < dividend_times[0]:
                optionValues[idx] = max(0,z*(price-X))
                optionValues[idx] = max(optionValues[idx],
                                        discount*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)]))
            else:
                no_exer = bt_american_discrete(price-dividend_amounts[0], X,
                                               T-dividend_times[0]*dt, r,
                                               sigma, N-dividend_times[0], option,
                                               [x-dividend_times[0] for x in dividend_times[1:nDiv]],
                                               dividend_amounts[1:nDiv])
                exer = max(0,z*(price-X))
                optionValues[idx] = max(no_exer,exer)

    return optionValues[0]

In [9]:
S0 = 165
X = 165
T = (datetime(2022,4,15)-datetime(2022,3,13)).days/365
r = 0.0425
q = 0.0053
sigma = 0.2
N = 200
dividend_times = [round((datetime(2023,4,11)-datetime(2023,3,13)).days/(datetime(2023,4,15)-datetime(2023,3,13)).days*N)]
dividend_amounts = [0.88]

call_w_div = bt_american_discrete(S0, X, T, r, sigma, N, "call", dividend_times, dividend_amounts)
put_w_div = bt_american_discrete(S0, X, T, r, sigma, N, "put", dividend_times, dividend_amounts)
call_wo_div = bt_american_cont(S0, X, T, r, q, sigma, N, "call")
put_wo_div = bt_american_cont(S0, X, T, r, q, sigma, N, "put")
print("the call with dividend:", call_w_div)
print("the call without dividend:", call_wo_div)
print("the put with dividend:", put_w_div)
print("the put without dividend:", put_wo_div)

the call with dividend: 4.120022561302723
the call without dividend: 4.227505716690514
the put with dividend: 4.109950677609662
the put without dividend: 3.714324427314217


In [10]:
def fd_greeks_div(S0,X,T,r,sigma,N,dividend_times,dividend_amounts,d=0.0001*S0,delta_only=False):
    if delta_only == True:
        delta_call = (bt_american_discrete(S0+d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts)-bt_american_discrete(S0-d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts))/2/d
        delta_put = (bt_american_discrete(S0+d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts)-bt_american_discrete(S0-d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts))/2/d
        return delta_call, delta_put
    delta_call = (bt_american_discrete(S0+d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts)-bt_american_discrete(S0-d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts))/2/d
    delta_put = (bt_american_discrete(S0+d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts)-bt_american_discrete(S0-d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts))/2/d
    gamma_call = (bt_american_discrete(S0+d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts)+bt_american_discrete(S0-d,X,T,r,sigma,N,'call',dividend_times, dividend_amounts)-2*bt_american_discrete(S0,X,T,r,sigma,N,'call',dividend_times, dividend_amounts))/d/d
    gamma_put = (bt_american_discrete(S0+d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts)+bt_american_discrete(S0-d,X,T,r,sigma,N,'put',dividend_times, dividend_amounts)-2*bt_american_discrete(S0,X,T,r,sigma,N,'put',dividend_times, dividend_amounts))/d/d
    vega_call = (bt_american_discrete(S0,X,T,r,sigma+d,N,'call',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T,r,sigma-d,N,'call',dividend_times, dividend_amounts))/2/d
    vega_put = (bt_american_discrete(S0,X,T,r,sigma+d,N,'put',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T,r,sigma-d,N,'put',dividend_times, dividend_amounts))/2/d
    theta_call = -(bt_american_discrete(S0,X,T+d,r,sigma,N,'call',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T-d,r,sigma,N,'call',dividend_times, dividend_amounts))/2/d
    theta_put = -(bt_american_discrete(S0,X,T+d,r,sigma,N,'put',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T-d,r,sigma,N,'put',dividend_times, dividend_amounts))/2/d
    rho_call = (bt_american_discrete(S0,X,T,r+d,sigma,N,'call',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T,r-d,sigma,N,'call',dividend_times, dividend_amounts))/2/d
    rho_put = (bt_american_discrete(S0,X,T,r+d,sigma,N,'put',dividend_times, dividend_amounts)-bt_american_discrete(S0,X,T,r-d,sigma,N,'put',dividend_times, dividend_amounts))/2/d
    sens_to_div_call = (bt_american_discrete(S0,X,T,r,sigma,N,'call',dividend_times,[dividend_amounts[0]+d])-bt_american_discrete(S0,X,T,r,sigma,N,'call',dividend_times, [dividend_amounts[0]-d]))/2/d
    sens_to_div_put = (bt_american_discrete(S0,X,T,r,sigma,N,'put',dividend_times,[dividend_amounts[0]+d])-bt_american_discrete(S0,X,T,r,sigma,N,'put',dividend_times, [dividend_amounts[0]-d]))/2/d
    return delta_call,delta_put,gamma_call,gamma_put,vega_call,vega_put,theta_call,theta_put,rho_call,rho_put,sens_to_div_call,sens_to_div_put

delta_call,delta_put,gamma_call,gamma_put,vega_call,vega_put,theta_call,theta_put,rho_call,rho_put,sens_to_div_call,sens_to_div_put = fd_greeks_div(S0,X,T,r,sigma,N,dividend_times,dividend_amounts,d=0.0001*S0)

In [11]:
print('Delta (Call):', delta_call)
print('Delta (Put):', delta_put)
print('Gamma (Call):', gamma_call)
print('Gamma (Put):', gamma_put)
print('Vega (Call):', vega_call)
print('Vega (Put):', vega_put)
print('Theta (Call):', theta_call)
print('Theta (Put):', theta_put)
print('Rho (Call):', rho_call)
print('Rho (Put):', rho_put)
print('Sensitivity to Dividend (Call):', sens_to_div_call)
print('Sensitivity to Dividend (Put):', sens_to_div_put)

Delta (Call): 0.5385793188325028
Delta (Put): -0.4930985911088082
Gamma (Call): -6.524726682829202e-12
Gamma (Put): -1.3049453365658404e-11
Vega (Call): 19.51481157714382
Vega (Put): 19.824543622322057
Theta (Call): -24.886811731374415
Theta (Put): -18.624780345836324
Rho (Call): 6.829726328485835
Rho (Put): -7.221868026267432
Sensitivity to Dividend (Call): -0.09355484175540854
Sensitivity to Dividend (Put): 0.5125798231177082


## Problem 2

In [12]:
os.chdir('/Users/xinyiwan/Downloads/fintech_545/Week7/Project')
port = pd.read_csv("problem2.csv")

In [13]:
def implied_vol_bt(S0, X, T, r, N, price, option, dividend_times=None, dividend_amounts=None):
    f = lambda sigma: (bt_american_discrete(S0, X, T, r, sigma, N, option, dividend_times, dividend_amounts)-price)
    return fsolve(f, x0 = 0.2, maxfev=10000)[0]

def calculate_portfolio_value(port, S0, r, b, current_date, Ss):
    port["ExpirationDate"] = pd.to_datetime(port["ExpirationDate"])
    for i in range(len(port)):
        X, price = port.loc[i,["Strike","CurrentPrice"]]
        T = (port.loc[i,"ExpirationDate"] - current_date).days/365
        if not pd.isna(port.loc[i,"ExpirationDate"]):
            dividend_times = [round((datetime(2023,3,15) - current_date).days/(port.loc[i,"ExpirationDate"] - current_date).days * N)]
        # dividend_amounts = [1.0]

        if port.loc[i,"OptionType"] in ["Call","Put"]:
            port.loc[i,"Implied Volatility"] = implied_vol_bt(S0, X, T, r, N, price, port.loc[i,"OptionType"].lower(), dividend_times, [1.0])
        else:
            port.loc[i,"Implied Volatility"] = 'NaN'
    
    d_values = {}
    for p in port['Portfolio'].unique().tolist():
        d_values[p] = {}
    
    for i in range(len(Ss)):
        S = Ss[i]
        for j in range(len(port)):
            X,sigma,holding,option_type,p = port.loc[j,["Strike", "Implied Volatility", "Holding", "OptionType","Portfolio"]]
            if option_type in ["Call", "Put"]:
                value = bt_american_discrete(S, X, T, r, sigma, N, option_type.lower(), dividend_times, [1.0]) * holding
            else:
                value = S * holding
            d_values[p][S] = d_values[p].get(S,0) + value
    return d_values

In [14]:
S0 = 151.03
r = 0.0425
current_date = datetime(2022,3,3)
N = 50
dividend_amounts = [1.0]
Ss = [151.03]
sigma = 0.2

calculate_portfolio_value(port, S0, r, b, current_date, Ss)

{'Straddle': {151.03: 12.387000074860097},
 'SynLong': {151.03: 2.687000074860098},
 'CallSpread': {151.03: 5.327000074859765},
 'PutSpread': {151.03: 3.010000000000044},
 'Stock': {151.03: 151.03},
 'Call ': {151.03: 7.537000074860098},
 'Put ': {151.03: 4.85},
 'CoveredCall': {151.03: 146.98000000000002},
 'ProtectedPut': {151.03: 154.03999999999996}}

In [15]:
daily_prices = pd.read_csv('DailyPrices.csv')
returns = library.return_calculate(daily_prices['AAPL'],method='LOG')
returns = [i for i in returns - returns.mean()]

In [16]:
def simulate_prices(returns,current_price,num_step,num_sim=1000):
    mu, std = norm.fit(returns)
    sim_returns = np.random.normal(mu, std, (num_sim, num_step))
    sim_prices = current_price * np.exp(sim_returns.cumsum(axis=1))
    return sim_prices, sim_returns

random.seed(99)
simulated_prices, simulated_returns = simulate_prices(returns, S0, 10, 500)
simulated_prices = simulated_prices[:,-1]

In [17]:
from tqdm import tqdm
num_sim = 500
portfolios = np.zeros((len(port['Portfolio'].unique().tolist()),num_sim))
for i, price in tqdm(enumerate(simulated_prices)):
    d_values = calculate_portfolio_value(port, S0, r, b, datetime(2023,3,13), Ss=[price])
    d_values = {key: value[next(iter(value))] for key, value in d_values.items()} # flatten dict values
    for j, key in enumerate(d_values.keys()):
        portfolios[j, i] = d_values[key]
portfolios.shape

500it [32:11,  3.86s/it]


(9, 500)

In [18]:
port

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.331474
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.264737
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.331474
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.264737
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.331474
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21,0.280299
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.264737
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84,0.296847
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03,NaN
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.331474


In [19]:
curr = [151.03]
curr_port = calculate_portfolio_value(port, S0, r, b, datetime(2023,3,3), Ss=curr)
curr_port = {key: value[next(iter(value))] for key, value in curr_port.items()}

df = pd.DataFrame.from_dict(curr_port, orient='index', columns=['Price'])
df_portfolios = pd.DataFrame(portfolios, index=df.index)
df = pd.concat([df, df_portfolios], axis=1)
df = pd.DataFrame(df.iloc[:,1:].values - df.iloc[:,0].values[:, np.newaxis], index=df.index)
df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
Straddle,0.023475,0.758796,1.356111,0.022656,15.609971,8.466887,1.404113,7.549178,0.622035,0.488399,...,2.673156,10.528453,17.814961,0.326414,2.362625,1.987542,0.384098,0.118968,0.378697,0.558542
SynLong,-0.750863,-6.472665,-8.439929,0.144276,24.991717,16.984732,-8.585092,15.864308,-5.957079,-5.449971,...,-11.470003,19.445235,27.315073,1.832455,7.988310,-10.047249,2.145207,0.692515,-5.031101,-5.716786
CallSpread,-0.161516,-1.548486,-2.013663,0.045325,4.808948,3.911856,-2.046655,3.718964,-1.419997,-1.324982,...,-2.601872,4.221024,5.007857,0.587392,2.108797,-2.330725,0.663599,0.220727,-1.247572,-1.374225
PutSpread,0.228643,1.861302,2.391296,-0.030951,-2.890562,-2.583636,2.434424,-2.513230,1.714004,1.547772,...,3.270035,-2.721720,-2.935935,-0.368125,-1.635361,2.890280,-0.437825,-0.141966,1.410117,1.635297
Stock,-0.746894,-6.320916,-8.163314,0.140900,25.846943,17.542249,-8.293982,16.359113,-5.811448,-5.309179,...,-11.031421,20.124878,28.192312,1.817720,8.103985,-9.671028,2.129555,0.683486,-4.893389,-5.573677
Call,-0.363694,-2.856934,-3.541909,0.083466,20.300844,12.725809,-3.590489,11.706743,-2.667522,-2.480786,...,-4.398424,14.986844,22.565017,1.079434,5.175468,-4.029854,1.264653,0.405742,-2.326202,-2.579122
Put,0.387169,3.615731,4.898020,-0.060810,-4.690873,-4.258923,4.994602,-4.157565,3.289557,2.969185,...,7.071580,-4.458391,-4.750056,-0.753020,-2.812843,6.017396,-0.880554,-0.286773,2.704899,3.137664
CoveredCall,-0.469424,-4.269390,-5.679531,0.088556,7.619172,6.565666,-5.785947,6.315775,-3.897825,-3.531509,...,-8.015321,6.985719,7.788781,1.023268,3.929921,-6.907415,1.184829,0.429572,-3.228264,-3.724413
ProtectedPut,-0.518739,-3.706325,-4.581818,0.098018,22.912446,14.833135,-4.643185,13.708056,-3.462663,-3.222198,...,-5.681366,17.310315,25.228098,1.266301,6.246946,-5.186555,1.483709,0.475722,-3.011486,-3.348884


In [20]:
mean = df.mean(axis=1)
var = df.apply(library.var,axis=1)
es = df.apply(library.es,axis=1)

result = pd.concat([mean, var, es], axis=1)
result.columns = ['Mean', 'VaR', 'ES']
result

,Mean,VaR,ES
Straddle,2.934361,-0.041406,-0.021459
SynLong,0.099347,17.599745,21.093383
CallSpread,0.148110,3.546573,3.887153
PutSpread,0.439253,2.620574,2.767577
Stock,0.380886,16.671101,19.902287
Call,1.516854,5.631834,6.028156
Put,1.417507,4.312130,4.519575
CoveredCall,-1.142948,13.042462,16.098311
ProtectedPut,1.613749,7.177510,7.603287


In [21]:
for i in range(len(port)):
    X, price = port.loc[i,["Strike","CurrentPrice"]]
    if price != 151.03:
        sigma = port.loc[i,"Implied Volatility"]
        if port.loc[i,"OptionType"]=="Call":
            port.loc[i,"delta"] = fd_greeks_div(S0,X,T,r,sigma,N,dividend_times,dividend_amounts,d=0.0001*S0,delta_only=True)[0]
        else:
            port.loc[i,"delta"] = fd_greeks_div(S0,X,T,r,sigma,N,dividend_times,dividend_amounts,d=0.0001*S0,delta_only=True)[1]
    else:
        port.loc[i,"delta"] = 1
port

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility,delta
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.29132,0.590677
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.239085,-0.413340
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.29132,0.590677
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.239085,-0.413340
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.29132,0.590677
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21,0.247297,0.268529
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.239085,-0.413340
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84,0.266725,-0.141236
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03,NaN,1.000000
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.29132,0.590677


In [57]:
current_price = 151.03
R_sigs,pvs = [],[]
P = port['Portfolio'].unique().tolist()
for i in range(len(P)):
    pi = port[port['Portfolio']==P[i]]
    pv = sum(pi['CurrentPrice']*pi['Holding'])
    pvs.append(pv)
    R_sig = abs(current_price/pv*sum(pi['Holding']*pi['delta']))*np.std(returns)*np.sqrt(10)
    R_sigs.append(R_sig)
var2,es2 = [],[]
for i in range(len(P)):
    var2.append(-pvs[i]*norm.ppf(0.05)*R_sigs[i])
    es2.append((pvs[i]*R_sigs[i]/0.05)*norm.pdf(norm.ppf(0.05)))
pd.DataFrame({'VaR':var2,'ES':es2},index = P)

,VaR,ES
Straddle,3.117919,3.909997
SynLong,17.652508,22.136957
CallSpread,5.663956,7.102830
PutSpread,4.784100,5.999454
Stock,17.581895,22.048405
Call,10.385214,13.023477
Put,7.267294,9.113480
CoveredCall,11.098402,13.917844
ProtectedPut,12.222024,15.326912


## Problem 3

In [58]:
FFR = pd.read_csv("F-F_Research_Data_Factors_daily.csv", parse_dates=['Date']).set_index('Date')
FFM = pd.read_csv("F-F_momentum_Factor_daily.csv", parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})
daily_prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
daily_returns = pd.DataFrame(library.return_calculate_adv(daily_prices)).set_index('Date')
daily_returns

,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,BRK-B,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.016127,0.023152,0.018542,0.008658,0.053291,0.007987,0.008319,0.015158,0.091812,0.006109,...,0.012807,-0.004082,0.004592,0.052344,0.003600,-0.012275,0.033021,0.026240,0.028572,0.013237
2022-02-16,0.001121,-0.001389,-0.001167,0.010159,0.001041,0.008268,0.007784,-0.020181,0.000604,-0.001739,...,0.006757,-0.002429,0.005763,0.038879,0.009294,0.012244,0.003363,0.015301,-0.001389,-0.025984
2022-02-17,-0.021361,-0.021269,-0.029282,-0.021809,-0.050943,-0.037746,-0.037669,-0.040778,-0.075591,-0.006653,...,-0.034949,0.005326,0.015017,-0.046988,-0.009855,0.004833,-0.030857,-0.031925,-0.033380,-0.028763
2022-02-18,-0.006475,-0.009356,-0.009631,-0.013262,-0.022103,-0.016116,-0.013914,-0.007462,-0.035296,0.003987,...,-0.000646,-0.000908,0.007203,-0.000436,-0.003916,-0.005942,-0.013674,-0.004506,-0.003677,0.015038
2022-02-22,-0.010732,-0.017812,-0.000729,-0.015753,-0.041366,-0.004521,-0.008163,-0.019790,-0.010659,-0.002033,...,0.009494,0.007121,-0.008891,0.003243,-0.001147,-0.000673,0.008342,-0.037654,-0.002246,-0.013605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-03,-0.010629,0.024400,-0.023621,-0.084315,0.009083,-0.027474,-0.032904,-0.011866,-0.028053,-0.010742,...,-0.004694,-0.011251,-0.001277,-0.002677,0.038211,0.004134,0.002336,-0.008916,-0.005954,0.001617
2023-02-06,-0.006111,-0.017929,-0.006116,-0.011703,0.025161,-0.017942,-0.016632,-0.002520,-0.000521,-0.000259,...,-0.014451,0.003945,0.001066,-0.007102,0.022012,0.021826,-0.041181,0.005106,-0.009782,-0.004595
2023-02-07,0.013079,0.019245,0.042022,-0.000685,0.010526,0.046064,0.044167,0.029883,0.051401,0.014720,...,-0.000368,-0.016473,-0.008518,0.019544,-0.003590,-0.001641,0.003573,0.001451,0.008669,-0.003618


In [59]:
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
df_factor = (FFR.join(FFM) / 100).loc['2013-1-31':]
all_data = daily_returns[stocks].join(df_factor)
data = all_data.dropna()

In [60]:
X = data[factors]
X = sm.add_constant(X)
y = data[stocks].sub(data['RF'],axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['alpha'])

for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']
alphas

,alpha
AAPL,0.000061
META,-0.000383
UNH,0.000588
MA,0.000393
MSFT,0.000087
NVDA,0.000404
HD,0.000091
PFE,-0.000053
AMZN,-0.000683
BRK-B,0.000136


In [61]:
td = 252
sub_returns = df_factor[factors] @ betas.T
sub_returns.columns = betas.index

merged_returns = pd.merge(sub_returns, df_factor['RF'], left_index=True, right_index=True)
merged_returns = merged_returns.add(merged_returns['RF'], axis=0).drop('RF', axis=1)
merged_returns = merged_returns.add(alphas.T.loc['alpha'], axis=1)

cumulative_returns = (merged_returns + 1).cumprod().tail(1)
daily_return_periods = merged_returns.shape[0]
expected_annual_return = ((cumulative_returns ** (1/daily_return_periods)) - 1) * td
expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [62]:
cov = all_data[stocks].cov()*td
cov

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.126877,0.139557,0.037447,0.081272,0.102937,0.171265,0.066193,0.032745,0.122117,0.055520,0.036945,0.037700,0.154880,0.058646,0.071406,0.087399,0.111906,0.022740,0.066245,0.066123
META,0.139557,0.400843,0.017102,0.102465,0.142255,0.240599,0.098845,0.045091,0.194794,0.061619,0.033637,0.020759,0.173121,0.073973,0.085022,0.127047,0.182074,0.021329,0.088791,0.076719
UNH,0.037447,0.017102,0.060922,0.031117,0.036318,0.046531,0.026045,0.032068,0.034737,0.028173,0.027861,0.026843,0.039128,0.033321,0.029590,0.022467,0.029806,0.022981,0.034744,0.028847
MA,0.081272,0.102465,0.031117,0.095762,0.079856,0.137369,0.056792,0.033440,0.096194,0.047520,0.031163,0.030837,0.097521,0.058309,0.082400,0.076987,0.079016,0.017382,0.063399,0.051734
MSFT,0.102937,0.142255,0.036318,0.079856,0.127839,0.175956,0.070916,0.035082,0.133856,0.052676,0.033859,0.031304,0.131911,0.056714,0.068230,0.088227,0.120259,0.020323,0.065226,0.060804
NVDA,0.171265,0.240599,0.046531,0.137369,0.175956,0.403814,0.112055,0.046362,0.221364,0.084290,0.041944,0.054455,0.291392,0.098399,0.118145,0.157837,0.188012,0.021758,0.113397,0.098663
HD,0.066193,0.098845,0.026045,0.056792,0.070916,0.112055,0.097074,0.033271,0.096833,0.042310,0.034414,0.015936,0.077754,0.043555,0.050164,0.063936,0.069626,0.022364,0.046487,0.048412
PFE,0.032745,0.045091,0.032068,0.033440,0.035082,0.046362,0.033271,0.070517,0.037274,0.031202,0.027470,0.019979,0.022283,0.031862,0.030650,0.024522,0.029355,0.027603,0.030812,0.029531
AMZN,0.122117,0.194794,0.034737,0.096194,0.133856,0.221364,0.096833,0.037274,0.242775,0.065772,0.030153,0.037446,0.187717,0.070819,0.083373,0.124253,0.149924,0.022616,0.085241,0.071419
BRK-B,0.055520,0.061619,0.028173,0.047520,0.052676,0.084290,0.042310,0.031202,0.065772,0.050175,0.024839,0.034026,0.061541,0.047076,0.042080,0.051383,0.056281,0.019263,0.050820,0.040421


In [63]:
def super_efficient_portfolio(returns, rf, cov_matrix):
    num_assets = returns.shape[1] if len(returns.shape) > 1 else returns.shape[0]
    
    def neg_sharpe_ratio(weights):
        port_return = returns.dot(weights)
        port_std_dev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
        sharpe = (port_return - rf) / port_std_dev
        return -sharpe
    
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'ineq', 'fun': lambda w: w}]
    bounds = [(0, 1) for _ in range(num_assets)]
    
    init_weights = np.ones(num_assets) / num_assets
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    opt_weights = opt_result.x
    opt_port_return = returns.dot(opt_weights)
    opt_port_std_dev = np.sqrt(opt_weights.T.dot(cov_matrix).dot(opt_weights))
    opt_sharpe_ratio = (opt_port_return - rf) / opt_port_std_dev
    
    return opt_weights*100, opt_sharpe_ratio

In [64]:
weights, sharpe_ratio = super_efficient_portfolio(expected_annual_return.values[0], 0.0425, cov)
print("Sharpe ratio:", sharpe_ratio)
weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight']).round(3)
weights.round(3)

Sharpe ratio: 1.470705216837335


,weight
AAPL,0.000
META,0.000
UNH,22.574
MA,0.000
MSFT,0.000
NVDA,0.000
HD,0.000
PFE,0.000
AMZN,0.000
BRK-B,0.000
